## Imports

In [1]:
import gdown
import os
import pathlib
import re
import shutil
import sys

from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Used for multiprocessing scripts
NUM_PROC = 20

#### Add parent directory to path to enable access to other scripts)

In [2]:
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Download Rellis-3D Data
Change dataset_path if you want to download to a different directory

In [3]:
dataset_path = os.path.join(os.getcwd(), "..", "datasets", "Rellis")
print(dataset_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis


## Download Images

In [4]:
rellis_images_path = os.path.join(dataset_path, "Rellis_3D_pylon_camera_node.zip")
print(rellis_images_path)
gdown.download("https://drive.google.com/uc?id=1F3Leu0H_m6aPVpZITragfreO_SGtL2yV", rellis_images_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\Rellis_3D_pylon_camera_node.zip


## Download Labels

In [5]:
rellis_labels_path = os.path.join(dataset_path, "Rellis_3D_pylon_camera_node_label_color.zip")
print(rellis_labels_path)
gdown.download("https://drive.google.com/uc?id=1HJl8Fi5nAjOr41DPUFmkeKWtDXhCZDke", rellis_labels_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\Rellis_3D_pylon_camera_node_label_color.zip


## Unzip images and labels

In [6]:
rellis_extract_path = os.path.join(dataset_path, "extracted")
print(rellis_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted


In [7]:
rellis_images_extract_path = os.path.join(rellis_extract_path, "images")
print(rellis_images_extract_path)
gdown.extractall(rellis_images_path, rellis_images_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted\images


['d:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/frame001353-1581624788_049.jpg',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/frame002181-1581624870_849.jpg',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/frame001253-1581624778_049.jpg',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/frame002652-1581624917_949.jpg',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\images\\Rellis-3D/00000/pylon_camera_node/frame00

In [8]:
rellis_labels_extract_path = os.path.join(rellis_extract_path, "labels")
print(rellis_labels_extract_path)
gdown.extractall(rellis_labels_path, rellis_labels_extract_path)

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\extracted\labels


['d:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\labels\\Rellis-3D/00000/pylon_camera_node_label_color/',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\labels\\Rellis-3D/00000/pylon_camera_node_label_color/frame000364-1581624689_150.png',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\labels\\Rellis-3D/00000/pylon_camera_node_label_color/frame001681-1581624820_850.png',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\labels\\Rellis-3D/00000/pylon_camera_node_label_color/frame000747-1581624727_450.png',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\extracted\\labels\\Rellis-3D/00000/pylon_camera_node_label_color/frame001069-1581624759_650.png',
 'd:\\Github Repos\\SJSU\\Offroad-Panoptic-Segmentation\\notebooks\\..\\datasets\\Rellis\\

# Clean Up File Structure

### Helper Methods

In [9]:
def split_name(filename):
    parts = re.split(r"\.|/|\\", str(filename))
    return (parts[-2], parts[-1])

def consolidate_files(dataset_dir, new_dir, file_ext):
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
        files = list(pathlib.Path(dataset_dir).glob(f"**/*.{file_ext}"))
        print(f"{len(files)} files with extension .{file_ext} found")
        for f in tqdm(files):
            file_name = ".".join(split_name(f))
            shutil.move(f, os.path.join(new_dir, file_name))
    else:
        print(f"Directory already exists at path: {new_dir}")

def remove_unpaired_images(image_dir, label_dir):
    if not os.path.exists(image_dir) or not os.path.exists(label_dir):
        return
    
    image_set = set(split_name(image)[0] for image in pathlib.Path(image_dir).glob("*"))
    label_set = set(split_name(label)[0] for label in pathlib.Path(label_dir).glob("*"))
    unpaired_images = image_set.difference(label_set)
    print(f"Deleting {len(unpaired_images)} unpaired images")

    for im in unpaired_images:
        image_path = os.path.join(image_dir, f"{im}.jpg")
        if os.path.exists(image_path):
            os.remove(image_path)

In [10]:
rellis_processed_images_path = os.path.join(dataset_path, "images")
print(rellis_processed_images_path)
rellis_processed_labels_path = os.path.join(dataset_path, "labels")
print(rellis_processed_labels_path)

consolidate_files(rellis_extract_path, rellis_processed_images_path, "jpg")
consolidate_files(rellis_extract_path, rellis_processed_labels_path, "png")

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\images
d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels
13556 files with extension .jpg found


100%|██████████| 13556/13556 [00:09<00:00, 1428.36it/s]


6234 files with extension .png found


100%|██████████| 6234/6234 [00:04<00:00, 1520.89it/s]


In [11]:
remove_unpaired_images(rellis_processed_images_path, rellis_processed_labels_path)

Deleting 7322 unpaired images


### For splitting data

In [12]:
def get_data_splits(image_dir, test_size=0.10, val_size=0.05, random_state=123):
    files = [split_name(filename)[0] for filename in pathlib.Path(image_dir).glob("*")]
    actual_test_size = val_size + test_size
    actual_val_size  = val_size / actual_test_size
    train, test = train_test_split(files, test_size=actual_test_size, random_state=random_state)
    test, val   = train_test_split(test,  test_size=actual_val_size,  random_state=random_state)
    print(len(train), len(test), len(val))
    return train, test, val

def make_data_dirs(image_dir, label_dir):
    for dir_name in [image_dir, label_dir]:
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        for split_name in ["train", "test", "val"]:
            split_dir = os.path.join(dir_name, split_name)
            if not os.path.exists(split_dir):
                os.mkdir(split_dir)

def move_files(train, test, val, image_dir, label_dir):
    make_data_dirs(image_dir, label_dir)

    for dir_name, data_split in [("train", train), ("test", test), ("val", val)]:
        for filename in tqdm(data_split, desc=dir_name):
            image_name = f"{filename}.jpg"
            label_name = f"{filename}.png"

            shutil.move(
                os.path.join(image_dir, image_name),
                os.path.join(image_dir, dir_name, image_name)
            )
            shutil.move(
                os.path.join(label_dir, label_name),
                os.path.join(label_dir, dir_name, label_name)
            )

In [13]:
train, test, val = get_data_splits(rellis_processed_images_path)

5298 624 312


In [14]:
move_files(train, test, val, rellis_processed_images_path, rellis_processed_labels_path)

val: 100%|██████████| 312/312 [00:00<00:00, 559.14it/s]


# Run scripts

## Generate Instance Ids

In [15]:
from createPanopticInstanceIds import main as cpii_main

cpii_main([rellis_processed_labels_path, "rellis", NUM_PROC])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6234/6234 [20:01<00:00,  5.19it/s]

6234 files generated
TOOK 1202.308479309082 SECONDS!


## Generate Panoptic Annotations

In [16]:
from createPanopticAnnotationsParallel import main as cpa_main

processed_labels_train_path = os.path.join(rellis_processed_labels_path, "train")
print(processed_labels_train_path)
cpa_main([processed_labels_train_path, "rellis", NUM_PROC])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\train


100%|██████████| 5298/5298 [01:48<00:00, 49.00it/s]



Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_train_panoptic.json
TOOK 109.36527276039124 SECONDS!


In [17]:
processed_labels_test_path = os.path.join(rellis_processed_labels_path, "test")
print(processed_labels_test_path)
cpa_main([processed_labels_test_path, "rellis", NUM_PROC])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\test


100%|██████████| 624/624 [00:13<00:00, 47.99it/s]



Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_test_panoptic.json
TOOK 13.34457516670227 SECONDS!


In [18]:
processed_labels_val_path = os.path.join(rellis_processed_labels_path, "val")
print(processed_labels_val_path)
cpa_main([processed_labels_val_path, "rellis", NUM_PROC])

d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\val


100%|██████████| 312/312 [00:07<00:00, 44.11it/s]



Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_val_panoptic.json
TOOK 7.36265230178833 SECONDS!


## Generate Instance Annotations

In [19]:
from createInstances import main as ci_main

ci_main([processed_labels_train_path, "rellis"])

Generating Panoptic Images: 100%|██████████| 5298/5298 [08:28<00:00, 10.42it/s]



Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_train_instances.json
TOOK 509.91044092178345 SECONDS!


In [20]:
ci_main([processed_labels_test_path, "rellis"])

Generating Panoptic Images: 100%|██████████| 624/624 [01:01<00:00, 10.12it/s]



Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_test_instances.json
TOOK 61.77594709396362 SECONDS!


In [21]:
ci_main([processed_labels_val_path, "rellis"])

Generating Panoptic Images: 100%|██████████| 312/312 [00:29<00:00, 10.46it/s]


Saving the json file d:\Github Repos\SJSU\Offroad-Panoptic-Segmentation\notebooks\..\datasets\Rellis\labels\annotations_val_instances.json
TOOK 29.898300170898438 SECONDS!


## Generate Semantic Segmentation Images

In [22]:
# Clone Panoptic API code
!git clone https://github.com/cocodataset/panopticapi.git ../panopticapi

Cloning into '../panopticapi'...


In [23]:
!python ../panopticapi/converters/panoptic2semantic_segmentation.py \
--input_json_file ../datasets/Rellis/labels/annotations_train_panoptic.json \
--segmentations_folder ../datasets/Rellis/labels/train_panoptic/ \
--semantic_seg_folder ../datasets/Rellis/labels/train_semantic \
--categories_json_file ../datasets/Rellis/labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: ../datasets/Rellis/labels/train_panoptic/
	JSON file: ../datasets/Rellis/labels/annotations_train_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: ../datasets/Rellis/labels/train_semantic
Creating folder ../datasets/Rellis/labels/train_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 265
Core: 0, 0 from 265 images processed
Core: 1, 0 from 265 images processed
Core: 2, 0 from 265 images processed
Core: 3, 0 from 265 images processed
Core: 4, 0 from 265 images processed
Core: 5, 0 from 265 images processed
Core: 6, 0 from 265 images processed
Core: 7, 0 from 265 images processed
Core: 8, 0 from 265 images processed
Core: 9, 0 from 265 images processed
Core: 10, 0 from 265 images processed
Core: 11, 0 from 265 images processed
Core: 12, 0 from 265 images processed
Core: 13, 0 from 265 images processed
Core: 14, 0 from 265 images processed
Core: 15, 0 from 2

In [24]:
!python ../panopticapi/converters/panoptic2semantic_segmentation.py \
--input_json_file ../datasets/Rellis/labels/annotations_test_panoptic.json \
--segmentations_folder ../datasets/Rellis/labels/test_panoptic/ \
--semantic_seg_folder ../datasets/Rellis/labels/test_semantic \
--categories_json_file ../datasets/Rellis/labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: ../datasets/Rellis/labels/test_panoptic/
	JSON file: ../datasets/Rellis/labels/annotations_test_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: ../datasets/Rellis/labels/test_semantic
Creating folder ../datasets/Rellis/labels/test_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 32
Core: 0, 0 from 32 images processed
Core: 1, 0 from 32 images processed
Core: 2, 0 from 32 images processed
Core: 3, 0 from 32 images processed
Core: 4, 0 from 31 images processed
Core: 5, 0 from 31 images processed
Core: 6, 0 from 31 images processed
Core: 7, 0 from 31 images processed
Core: 8, 0 from 31 images processed
Core: 9, 0 from 31 images processed
Core: 10, 0 from 31 images processed
Core: 11, 0 from 31 images processed
Core: 12, 0 from 31 images processed
Core: 13, 0 from 31 images processed
Core: 14, 0 from 31 images processed
Core: 15, 0 from 31 images processed
C

In [25]:
!python ../panopticapi/converters/panoptic2semantic_segmentation.py \
--input_json_file ../datasets/Rellis/labels/annotations_val_panoptic.json \
--segmentations_folder ../datasets/Rellis/labels/val_panoptic/ \
--semantic_seg_folder ../datasets/Rellis/labels/val_semantic \
--categories_json_file ../datasets/Rellis/labels/categories.json

EXTRACTING FROM...
COCO panoptic format:
	Segmentation folder: ../datasets/Rellis/labels/val_panoptic/
	JSON file: ../datasets/Rellis/labels/annotations_val_panoptic.json
SEMANTIC SEGMENTATION
in PNG format:
	Folder with semnatic segmentations: ../datasets/Rellis/labels/val_semantic
Creating folder ../datasets/Rellis/labels/val_semantic for semantic segmentation PNGs


Number of cores: 20, images per core: 16
Core: 0, 0 from 16 images processed
Core: 1, 0 from 16 images processed
Core: 2, 0 from 16 images processed
Core: 3, 0 from 16 images processed
Core: 4, 0 from 16 images processed
Core: 5, 0 from 16 images processed
Core: 6, 0 from 16 images processed
Core: 7, 0 from 16 images processed
Core: 8, 0 from 16 images processed
Core: 9, 0 from 16 images processed
Core: 10, 0 from 16 images processed
Core: 11, 0 from 16 images processed
Core: 12, 0 from 15 images processed
Core: 13, 0 from 15 images processed
Core: 14, 0 from 15 images processed
Core: 15, 0 from 15 images processed
Core: